In [ ]:
from pistonball_CleanRL import *
import torch
from supersuit import color_reduction_v0, frame_stack_v1, resize_v1
from pettingzoo.butterfly import pistonball_v6
import pandas as pd
import cv2
import os

In [ ]:
save_dir = "saved_models"
log_dir = "training_logs"
model_name = "model_episode_300.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 读取训练日志
log_path = os.path.join(log_dir, "training_log.csv")
logs_df = pd.read_csv(log_path)

# 打印日志
print(logs_df)

In [ ]:
""" RENDER THE POLICY """
env = pistonball_v6.parallel_env(render_mode="rgb_array", continuous=False)
env = color_reduction_v0(env)
env = resize_v1(env, 64, 64)
env = frame_stack_v1(env, stack_size=4)
num_actions = env.action_space(env.possible_agents[0]).n
render_episodes = 3

vedio_filename = "pistonball_v6.mp4"
vedio_path = os.path.join(log_dir, vedio_filename)
vedio_fps = 30
vedio_framesize = (env.unwrapped.screen_width, env.unwrapped.screen_height)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
vedio_out = cv2.VideoWriter(vedio_path, fourcc, vedio_fps, vedio_framesize)

# agent = Agent(num_actions=num_actions).to(device)
agent = Agent_ADG(num_actions=num_actions).to(device)
model_path = os.path.join(save_dir, model_name)

if model_name:
    agent.load_state_dict(torch.load(model_path, map_location=device))
    print("Model loaded successfully.")

agent.eval()

with torch.no_grad():
    for episode in range(render_episodes):
        obs, infos = env.reset(seed=None)
        obs = batchify_obs(obs, device)
        terms = [False]
        truncs = [False]
        while not any(terms) and not any(truncs):
            vedio_frame = env.render()
            vedio_out.write(vedio_frame)
            actions, logprobs, _, values = agent(obs)
            obs, rewards, terms, truncs, infos = env.step(unbatchify(actions, env))
            obs = batchify_obs(obs, device)
            terms = [terms[a] for a in terms]
            truncs = [truncs[a] for a in truncs]
vedio_out.release()